In [94]:
import pandas as pd

#### import contig-host info 

In [95]:
#import crispr results
crispr = pd.read_csv("../../tmp2/10_contaminants/pilecr/crispr.finalOutput.csv",header=0,sep='\t')
#import mvp results
mvp = pd.read_csv("../../tmp2/10_contaminants/MVP/mvp.finaloutput.csv",header=0,sep='\t')

#combine info
#cvh = pd.concat([crispr,mvp]).drop_duplicates() #cvh := contig versus host
#getName = lambda x : ' '.join(x.split(' ')[:2])
#cvh["name"] = cvh["hostName"].apply(getName)
#cvh = cvh[["qid","name"]].drop_duplicates()
#cvh.shape

In [96]:
#tax info for the crispr table was obtained from batch entrez
taxInfocrispr = pd.read_csv("../../tmp2/10_contaminants/pilecr/taxInfoCrisprTable.csv",header=0,sep='\t')
taxInfocrispr = dict(zip(taxInfocrispr["acc"].values,taxInfocrispr["taxid"]))
getTax = lambda x:taxInfocrispr[x]
crispr["taxid"] = crispr["qaccver"].apply(getTax)

In [97]:
#combine info crispr and mvp
crispr = crispr[["saccver","qaccver","lineage","name","taxid"]]
mvp = mvp[["qid","sid","lineage","hostName","host_taxon_id"]]
columns = ["contig","sid","lineage","hostName","host_taxon_id"]
crispr.columns = columns
mvp.columns = columns
output = pd.concat([crispr,mvp])

In [98]:
output.head()

,contig,sid,lineage,hostName,host_taxon_id
0,9_P1_8Lpp_out_L5890,CP001107.1,"['[Eubacterium] rectale ATCC 33656', 'Bacteria...",[Eubacterium] rectale ATCC 33656,515619
1,9_P1_8Lpp_out_L5890,FP929042.1,"['[Eubacterium] rectale DSM 17629', 'Bacteria'...",[Eubacterium] rectale DSM 17629,657318
2,983_Ra2_8Lpp_out_L551,CP002780.1,"['Desulfotomaculum ruminis DSM 2154', 'Bacteri...",Desulfotomaculum ruminis DSM 2154,696281
3,970_B2_8Lpp_out_L3046,CP010432.1,"['Lactobacillus acidophilus', 'Bacteria', 'Fir...",Lactobacillus acidophilus,1579
4,970_B2_8Lpp_out_L3046,CP005926.2,"['Lactobacillus acidophilus La-14', 'Bacteria'...",Lactobacillus acidophilus La-14,1314884


In [99]:
output.shape

(4998, 5)

In [100]:
#Number of contigs with host prediction
output["contig"].unique().shape

(910,)

In [101]:
#Number of strains found as hosts
output["sid"].unique().shape

(1280,)

In [102]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()

tree = ncbi.get_topology(output["host_taxon_id"].unique(),intermediate_nodes=True)

def getLineage(node):
    s = ""
    g = ""
    f = ""
    o = ""
    c = ""
    p = ""
    k = ""
    node = tree.search_nodes(name=node)[0]
    ancestors = node.get_ancestors()
    while len(ancestors) > 0:
        parent = ancestors[0]
        if str(parent.rank) == "species":
            s = str(parent.sci_name)
        elif str(parent.rank) == "genus":
            g = str(parent.sci_name)
        elif str(parent.rank) == "family":
            f = str(parent.sci_name)
        elif str(parent.rank) == "order":
            o = str(parent.sci_name)
        elif str(parent.rank) == "class":
            c = str(parent.sci_name)
        elif str(parent.rank) == "phylum":
            p = str(parent.sci_name)
        elif str(parent.rank) == "superkingdom":
            k = str(parent.sci_name)
        ancestors = parent.get_ancestors()
    return [s,g,f,o,c,p,k]

lineage = {}
for taxid in output["host_taxon_id"].unique():
    try:
        lineage[taxid] = getLineage(str(taxid))
    except:
        print taxid
        
f = lambda x:lineage[x]
output["lineage"] = output["host_taxon_id"].apply(f)

In [103]:
lineageDF = pd.DataFrame(lineage).T.reset_index()
lineageDF.columns = ["taxid","species","genus","family","order","class","phylum","kingdom"]

output = output.merge(right=lineageDF,how="inner",left_on="host_taxon_id",right_on="taxid")

In [104]:
output.shape

(4998, 13)

In [106]:
#Number of strains == number of accession numbers
output.sid.unique().shape

(1280,)

In [122]:
#Number of strains == number of tax id numbers
output[["genus","species","host_taxon_id"]].drop_duplicates().shape

(1147, 3)

In [116]:
#Number of species
output[["genus","species"]].drop_duplicates().shape

(187, 2)

In [127]:
output["genus"].unique().shape

(79,)

In [126]:
output[["family","genus"]].drop_duplicates().shape

(87, 2)

In [120]:
#Number of species per phylum
output[["phylum","genus","species"]].drop_duplicates().groupby("phylum").count()

,genus,species
phylum,,
Actinobacteria,11,11
Bacteroidetes,41,41
Chlorobi,1,1
Euryarchaeota,1,1
Firmicutes,92,92
Fusobacteria,1,1
Proteobacteria,38,38
Tenericutes,3,3
Verrucomicrobia,2,2


In [138]:
output[output["genus"]=="Bacteroides"].species.unique()

array(['Bacteroides dorei', '', 'Bacteroides fragilis',
       'Bacteroides vulgatus', 'Bacteroides plebeius',
       'Bacteroides caccae', 'Bacteroides cellulosilyticus',
       'Bacteroides stercoris', 'Bacteroides uniformis',
       'Bacteroides coprocola', 'Bacteroides massiliensis',
       'Bacteroides gallinarum', 'Bacteroides sartorii',
       'Bacteroides salyersiae', 'Bacteroides finegoldii',
       'Bacteroides nordii', 'Bacteroides ovatus',
       'Bacteroides xylanisolvens', 'Bacteroides thetaiotaomicron',
       'Bacteroides eggerthii', 'Bacteroides coprophilus',
       'Bacteroides clarus'], dtype=object)

In [ ]:
def getPhylumNode(node):
    parent = node.get_ancestors()[0]
    if str(parent.rank) == "species":
        return parent
    else:
        return getPhylumNode(parent)

In [34]:
x = tree.search_nodes(name="515619")[0]
print x.rank

no rank


In [47]:
x = x.get_ancestors()[0]
print x.rank

superkingdom


In [41]:
x.sci_name

u'Lachnospiraceae'

In [36]:
x.featur

{'common_name',
 'dist',
 'lineage',
 'name',
 'named_lineage',
 'rank',
 'sci_name',
 'support',
 'taxid'}

In [22]:
#contigs annotated as crAssphage
crass = pd.read_csv("../../tmp2/10_contaminants/crisprAndMvpExploration/crassContigs.txt",header=None)[0].values

In [24]:
contigs = {}
for x in output["contig"].values:
    o = x # o:= original name
    x = x.split('_')
    contigs[x[1]+"_contig"+x[0]] = o

In [27]:
for x in list(set(list(crass)) & set(contigs.keys())):
    print x,contigs[x]

T2_contig1542 1542_T2_8Lpp_out_L697
M1_contig2523 2523_M1_8Lpp_out_L9401


In [57]:
for x in output[output["contig"]=="2523_M1_8Lpp_out_L9401"]["lineage"]:
    print x

['Porphyromonas gingivalis TDC60', 'Bacteria', 'Bacteroidetes', 'Bacteroidia', 'Bacteroidales', 'Porphyromonadaceae', 'Porphyromonas']
['Porphyromonas gingivalis TDC60', 'Bacteria', 'Bacteroidetes', 'Bacteroidia', 'Bacteroidales', 'Porphyromonadaceae', 'Porphyromonas']


In [37]:
#contigs annotated as Eukaryotic viruses
crass = pd.read_csv("../../tmp2/10_contaminants/crisprAndMvpExploration/eukaryoticViruses.txt",header=None)[0].values

contigs = {}
for x in output["contig"].values:
    o = x # o:= original name
    x = x.split('_')
    contigs[x[1]+"_contig"+x[0]] = o

In [38]:
for x in list(set(list(crass)) & set(contigs.keys())):
    print x,contigs[x]

N1_contig102 102_N1_8Lpp_out_L2422
B2_contig627 627_B2_8Lpp_out_L3841
Da1_contig8010 8010_Da1_8Lpp_out_L1834
K1_contig9576 9576_K1_8Lpp_out_L2354
Q1_contig4952 4952_Q1_8Lpp_out_L739
X2_contig811 811_X2_8Lpp_out_L2904


In [53]:
for x in output[output["contig"]=="811_X2_8Lpp_out_L2904"]["lineage"]:
    print x

[u'root', u'cellular organisms', u'Bacteria', u'FCB group', u'Bacteroidetes/Chlorobi group', u'Bacteroidetes', u'Bacteroidia', u'Bacteroidales', u'Bacteroidaceae', u'Bacteroides', u'Bacteroides eggerthii', u'Bacteroides eggerthii DSM 20697']


In [54]:
#how many hosts
output["sid"].unique().shape

(1280,)

In [58]:
output.head()

,contig,sid,lineage,hostName,host_taxon_id
0,9_P1_8Lpp_out_L5890,CP001107.1,"['[Eubacterium] rectale ATCC 33656', 'Bacteria...",[Eubacterium] rectale ATCC 33656,515619
1,9_P1_8Lpp_out_L5890,FP929042.1,"['[Eubacterium] rectale DSM 17629', 'Bacteria'...",[Eubacterium] rectale DSM 17629,657318
2,983_Ra2_8Lpp_out_L551,CP002780.1,"['Desulfotomaculum ruminis DSM 2154', 'Bacteri...",Desulfotomaculum ruminis DSM 2154,696281
3,970_B2_8Lpp_out_L3046,CP010432.1,"['Lactobacillus acidophilus', 'Bacteria', 'Fir...",Lactobacillus acidophilus,1579
4,970_B2_8Lpp_out_L3046,CP005926.2,"['Lactobacillus acidophilus La-14', 'Bacteria'...",Lactobacillus acidophilus La-14,1314884


In [62]:
output[["contig","sid"]].drop_duplicates().groupby("contig").count()["sid"].describe()

count    910.000000
mean       3.286813
std        7.816807
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max      105.000000
Name: sid, dtype: float64

In [67]:
output[["contig","sid"]].drop_duplicates().groupby("sid").count().describe()

,contig
count,1280.000000
mean,2.336719
std,2.254675
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,30.000000
